In [3]:
import pandas as pd
import numpy as np
from scipy.special import factorial2

hbarc = 197.3 #MeV fm
hbar = 6.582E-22 #MeV s
alpha = 1/137 
mP = 938.272 # Mev/c^2
#muN = 3.152E-14 # MeV / T
muN2 =  alpha*hbarc**3*2*mP**-2 # nuclear magneton squared

def Thalf_width(E): #converts energy width (MeV) to Thalf in seconds
    return hbar * np.log(2)/E

def Thalf_width_kev(E):
    if E==0.:
        return None
    return Thalf_width(E/1000)

def times_so_far(ls):
    out = [0]*len(ls)
    for i in range(len(ls)):
        out[i] = ls[:i+1].count(ls[i])
    return out

def j_to_twoj(x):
    if '/2' in x:
        return int(x.strip('/2'))
    else:
        return 2*int(x.strip('()'))
    
def thalf_units_to_sec(units,value):
    m=0
    if units=='PS':
        m = 10**-12
    elif units=='NS':
        m = 10**-9
    elif units=='FS':
        m = 10**-15
    elif units=='US':
        m = 10**-6
    elif units=='AS':
        m = 10**-18
    elif units=='KEV':
        pass
    if m==0:
        return Thalf_width_kev(value)
    else:
        return m*value
    
def my_float(x):
    try:
        return float(x)
    except:
        return 0.0
    
def fLambda(l):
    return 8*np.pi*(l+1)/(l*(factorial2(2*l+1)**2))

# def BEl(l,Egamma,Thalf,I,units):
#     if units == 1: # [B] = fm^2l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#     elif units == 2: # [B] = e^2 fm^2l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#         B = B/(alpha*hbarc)
#     elif units == 3: # [B] = e^2 b^l
#         B = (I/100)*(np.log(2)/Thalf)*fLambda(l)**-1*hbar*hbarc**(2*l+1)*(Egamma**(-2*l-1))
#         B = 10**(-2*l)*B/(alpha*hbarc)
#     return B

def BM1(Egamma,Thalf):
    return (np.log(2)/Thalf) * (1/1.76E13) * Egamma**-3 

# def BE2W(A,Egamma,Thalf,I):
#     lamW = 7.3E7*A**(4/3)*Egamma**5;
#     lamExp = (np.log(2)/Thalf)*(I/100);
#     return lamExp/lamW

# def Bweisskopf(l,A):
#     # Weisskopf (single-particle) estimate in e^2 fm^2l
#     return (1/(4*np.pi)) * (3/(3+l))**2 * (1.2*A**(1/3))**(2*l)

# def BEl_in_Wu(l,Egamma,Thalf,I,A):
#     return BEl(l,Egamma,Thalf,I,2) / Bweisskopf(l,A)

# def BWu_convert(BEl_Wu,l,A):
#     return BEl_Wu*Bweisskopf(l,A)

BM1_Weisskopf = (10/np.pi)*0.75**2


In [6]:
# df = pd.read_csv('sd_E2_data.csv',sep=r'\s*,\s*',header=0, encoding='ascii', engine='python')
df = pd.read_csv('M1_prep.csv')
df = df.rename(columns=lambda x: x.strip())

In [7]:
def isNaN(num):
    return num != num

for i in df.index:
    if isNaN(df['A'].loc[i]):
#         print(df['A'].loc[i])
        df = df.drop(i)
    
df['A'] = df['A'].map(int)
df['Z'] = df['Z'].map(int)
df['N'] = df['N'].map(int)

In [8]:
# df['Element'] = df['Element'].apply(lambda x: str.strip(x))

for key in df:
    try:
        df[key] = df[key].apply(lambda x: str.strip(x))
    except:
        pass


df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df['Element'] = df['Element'].apply(str.capitalize)

In [5]:
df = df.rename(columns={'B(W.u.)':'B_exp (W.u.)'})
df = df.rename(columns={'B Unc (W.u.)':'B_exp Unc (W.u.)'})

In [6]:
df['Jpi'] = df['Jpi'].map(lambda x: str(x).strip().strip('()+'))
df = df.rename(columns={'Jpi':'Ji'})

df['twoJi'] = df['Ji'].map(j_to_twoj)

In [7]:
for i in df.index:
    string = df['B_exp Unc (W.u.)'].loc[i]
    try:
        up,down = string.strip('+').split('-')
        df['B_exp Unc (W.u.)'].loc[i] = max(float(up),float(down))
    except:
        pass
    if string=='':
        df['B_exp Unc (W.u.)'].loc[i] = 0.

C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
df

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),I_gamma,I_gamma Unc,Mult,B_exp (W.u.),B_exp Unc (W.u.),twoJi
0,16,O,8,8,9844.50,0.50,2,0.62 KEV 0.1,2927.1,0.8,34.0,7,[M1],0.0042,8.0E-4,4
1,18,O,8,10,3920.44,0.14,2,18.4 FS 2,1938.0,,87.6,0.7,M1,0.1400,0.02,4
2,18,O,8,10,5254.80,0.90,2,7.0 FS 0.3,1334.0,,8.7,0.4,M1,0.1110,0.008,4
3,18,O,8,10,7116.90,1.20,4,17 FS LT,3562.0,,69.2,0.7,M1,0.0710,0.016,8
4,18,O,8,10,8213.00,4.00,2,1.0 KEV 0.8,4293.0,,3.0,1,M1,0.0072,0.003,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,38,K,19,19,3429.70,1.00,2,0.12 PS LT,1028.0,1.2,100.0,5,[M1],0.1000,0.0,4
291,38,K,19,19,3855.80,1.90,1,83 FS LT,3725.4,1.9,100.0,,[M1],0.0051,0.0,2
292,38,K,19,19,3976.30,1.20,1,40 FS LT,3848.0,1.9,100.0,10,[M1],0.0071,0.0,2
293,39,K,19,20,4095.60,0.40,1/2,60 FS 10,1572.5,0.3,100.0,1.3,[M1],0.0800,0.018,1


In [9]:
df[['Thalf Value','Thalf Units','Thalf Unc']] = df['Thalf'].str.split(expand=True)

df = df[df['Thalf Unc']!='LT']
df = df[df['Thalf Unc']!='LE']
df = df[df['Thalf Unc']!='GT']
df = df[df['I_gamma Unc']!='AP']

df['Thalf Unc'] = df['Thalf Unc'].fillna(0.0)
df['E_gamma Unc (keV)'] = df['E_gamma Unc (keV)'].fillna(0.0)
df['I_gamma Unc'] = df['I_gamma Unc'].fillna(0.0)

for i in df.index:
    string = df['Thalf Unc'].loc[i]
    try:
        up,down = string.strip('+').split('-')
        df['Thalf Unc'].loc[i] = max(float(up),float(down))
    except:
        pass

df['Thalf (sec)'] = df[['Thalf Units','Thalf Value']].apply(lambda row: thalf_units_to_sec(row['Thalf Units'],float(row['Thalf Value'])),axis=1,result_type='expand')
df['Thalf Unc (sec)'] = df[['Thalf Units','Thalf Unc']].apply(lambda row: thalf_units_to_sec(row['Thalf Units'],float(row['Thalf Unc'])),axis=1,result_type='expand')

C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [10]:
for col in ['I_gamma','I_gamma Unc','E_gamma (keV)','E_gamma Unc (keV)']:
    df[col] = df[col].apply(my_float)

# df['B_exp (e^2fm^4)'] = df[['E_gamma (keV)','Thalf (sec)','I_gamma']].apply(lambda row: BEl(2,row['E_gamma (keV)']/1000,row['Thalf (sec)'],row['I_gamma'],2),axis=1,result_type='expand')

# NOTE: the 5 on the E_gamma term below comes from the derivative dB/dE and is 5 = |-2*lambda - 1|
# df['B_exp Unc (e^2fm^4)'] = df.apply(lambda row: row['B_exp (e^2fm^4)']*np.sqrt( (row['I_gamma Unc']/row['I_gamma'])**2 + (5 * row['E_gamma Unc (keV)']/row['E_gamma (keV)'])**2 + (row['Thalf Unc (sec)']/row['Thalf (sec)'])**2 ) , axis=1 ,result_type='expand' ) 

In [11]:
df['B_exp (mu_N^2)'] = -9.
df['B_exp Unc (mu_N^2)'] = -9.

df['B_exp Unc (W.u.)'] = df['B_exp Unc (W.u.)'][df['B_exp Unc (W.u.)'] != '<']
df['B_exp Unc (W.u.)'] = df['B_exp Unc (W.u.)'][df['B_exp Unc (W.u.)'] != '>']

for i in df.index:
#     print(df['B_exp (W.u.)'].loc[i])
#     print(df['B_exp Unc (W.u.)'].loc[i])
#     print(df['A'].loc[i])
    df['B_exp (mu_N^2)'].loc[i] = BM1_Weisskopf * float(df['B_exp (W.u.)'].loc[i])
    df['B_exp Unc (mu_N^2)'].loc[i] = BM1_Weisskopf * float(df['B_exp Unc (W.u.)'].loc[i])
    

In [12]:
df['Nucleus'] = df.apply(lambda x: str(int(x['A']))+x['Element'], axis=1)

In [13]:
df_levels = pd.read_csv('sd_levels_20mev.csv')


df_levels['E(keV)'] = None
df_levels['dE(keV)'] = None
    
for i in df_levels.index:
#     print(df_levels['Elevel(keV)'].iloc[i])
    E = df_levels['Elevel(keV)'].loc[i].split()
    if len(E)==1:
        df_levels['E(keV)'].loc[i] = float(E[0].replace('?',''))
        df_levels['dE(keV)'].loc[i] = None
    else:
        df_levels['E(keV)'].loc[i] = float(E[0].replace('?',''))
        df_levels['dE(keV)'].loc[i] = float(E[1].replace('?',''))  

df_levels = df_levels.drop('unclean_E',axis=1)
df_levels = df_levels.drop('T1/2',axis=1)
df_levels = df_levels.drop('unclean_Jpi',axis=1)
df_levels = df_levels.drop('unclean_T1/2',axis=1)
df_levels['Include'] = True
for char in [',','&','-','GE',':','TO']:
    df_levels['Include'] *= df_levels['Jpi'].map(lambda x : not char in str(x))
df_levels = df_levels.drop(df_levels[df_levels['Include']==False].index)
for char in ['+','(',')','[',']']:
    df_levels['Jpi'] = df_levels['Jpi'].map(lambda x: str(x).replace(char,''))
df_levels = df_levels.rename(columns={'Jpi':'J'})
df_levels = df_levels.drop(df_levels[df_levels['J']==''].index)
df_levels['twoJ'] = df_levels['J'].map(j_to_twoj)

df_levels['Element'] = df_levels['Nucleus'].map(lambda x : x[2:])
df_levels['Element'] = df_levels['Element'].apply(str.capitalize)
df_levels['A'] = df_levels['Nucleus'].map(lambda x : x[:2])
df_levels['Nucleus'] = df_levels.apply(lambda x: str(int(x['A']))+x['Element'], axis=1)

C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\computation\expressions.py:204: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  warnings.warn(


In [14]:
df_levels['Nucleus'].unique()

array(['11O', '12O', '14O', '15O', '16O', '17O', '18O', '19O', '20O',
       '21O', '22O', '23O', '24O', '26O', '15F', '16F', '17F', '18F',
       '19F', '20F', '21F', '22F', '23F', '24F', '25F', '26F', '27F',
       '16Ne', '18Ne', '19Ne', '20Ne', '21Ne', '22Ne', '23Ne', '24Ne',
       '25Ne', '26Ne', '27Ne', '28Ne', '29Ne', '30Ne', '32Ne', '34Ne',
       '17Na', '19Na', '20Na', '21Na', '22Na', '23Na', '24Na', '25Na',
       '26Na', '27Na', '28Na', '29Na', '30Na', '31Na', '33Na', '35Na',
       '20Mg', '21Mg', '22Mg', '23Mg', '24Mg', '25Mg', '26Mg', '27Mg',
       '28Mg', '29Mg', '30Mg', '31Mg', '32Mg', '34Mg', '36Mg', '38Mg',
       '40Mg', '21Al', '22Al', '23Al', '24Al', '25Al', '26Al', '27Al',
       '28Al', '29Al', '30Al', '31Al', '32Al', '33Al', '35Al', '39Al',
       '22Si', '23Si', '24Si', '25Si', '26Si', '27Si', '28Si', '29Si',
       '30Si', '31Si', '32Si', '33Si', '34Si', '35Si', '36Si', '37Si',
       '38Si', '39Si', '40Si', '42Si', '44Si', '25P', '26P', '27P', '28P',
     

In [15]:
df_levels['n'] = None
for nucleus in df_levels['Nucleus'].unique():
    ls = list(df_levels[df_levels['Nucleus']==nucleus]['twoJ'])
#     print(ls)
    tsf = times_so_far(ls)
    j = 0
    for i in df_levels[df_levels['Nucleus']==nucleus].index:
#         print(i)
        df_levels['n'].loc[i] = tsf[j]
        j+=1

C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [16]:
df_levels

,Nucleus,Elevel(keV),J,E(keV),dE(keV),Include,twoJ,Element,A,n
0,11O,0.49E3,5/2,490.0,None,True,5,O,11,1
1,11O,2.12E3,5/2,2120.0,None,True,5,O,11,2
2,12O,0.0,0,0.0,None,True,0,O,12,1
3,12O,1.62E3 11,0,1620.0,11.0,True,0,O,12,2
4,12O,1968 52,2,1968.0,52.0,True,4,O,12,1
...,...,...,...,...,...,...,...,...,...,...
7876,52Ca,2563 1,2,2563.0,1.0,True,4,Ca,52,1
7877,54Ca,0.0,0,0.0,None,True,0,Ca,54,1
7878,54Ca,2043 19,2,2043.0,19.0,True,4,Ca,54,1
7879,56Ca,0,0,0.0,None,True,0,Ca,56,1


In [27]:
df['Jf'] = None
df['twoJf'] = None
df['ni'] = None
df['nf'] = None
df['Include'] = True

max_deltaJ = 1
tol_energy = 0.5
tol_gamma = 0.5

# for i in df.index:
for i in df.index:
    nuc_cond = (df_levels['Nucleus']==df['Nucleus'].loc[i])
    i_J_cond = (df_levels['twoJ']==df['twoJi'].loc[i])
    i_E_cond = (tol_energy > abs(df_levels['E(keV)'] - df['Energy (keV)'].loc[i]))
    f_J_cond = ( 2*max_deltaJ >= abs(df_levels['twoJ'] - df['twoJi'].loc[i]))
    f_E_cond = (tol_gamma > abs(df_levels['E(keV)'] - df['Energy (keV)'].loc[i] + df['E_gamma (keV)'].loc[i]))
    i_condition =  nuc_cond &  i_J_cond  &  i_E_cond
    f_condition =  nuc_cond &  f_J_cond  &  f_E_cond
    state_i = df_levels[i_condition]
    state_f = df_levels[f_condition]
    
#     print('len=',len(state_i))
    
    if (len(state_i)!=1)|(len(state_f)!=1):
        df['Include'].loc[i] = False
        print(f"-------------------------------------------")
        print(f"Excluding transition i = {i}")
        print(f'Matched # initial states: {len(state_i)}' )
        print(f'Matched # final   states: {len(state_f)}' )
        print(f"Matched # Ji             : { len( df_levels[ nuc_cond & i_J_cond ] ) }")
        print(f"Matched # Jf             : { len( df_levels[ nuc_cond & f_J_cond ] ) }")
        print(f"Matched # Ei             : { len( df_levels[ nuc_cond & i_E_cond ] ) }")
        print(f"Matched # Ef             : { len( df_levels[ nuc_cond & f_E_cond ] ) }")
        print(state_i)
        print(state_f)
        print(df.loc[i])

        continue
        
    df['Jf'].loc[i] = list(state_f['J'])[0]
    df['twoJf'].loc[i] = int(state_f['twoJ'])
    df['ni'].loc[i] = int(state_i['n'])
    df['nf'].loc[i] = int(state_f['n'])
    
#     print(state_i)
#     print(state_f)
#     print(df.loc[i])
    

C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

-------------------------------------------
Excluding transition i = 7
Matched # initial states: 1
Matched # final   states: 0
Matched # Ji             : 3
Matched # Jf             : 13
Matched # Ei             : 1
Matched # Ef             : 0
    Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
248     18F   1041.55 8  0  1041.55     8.0     True     0       F  18  1
Empty DataFrame
Columns: [Nucleus, Elevel(keV), J, E(keV), dE(keV), Include, twoJ, Element, A, n]
Index: []
A                               18
Element                          F
Z                                9
N                                9
Energy (keV)               1041.55
Energy Unc (keV)              0.08
Ji                               0
Thalf                 1.77 FS 0.31
E_gamma (keV)               1041.0
E_gamma Unc (keV)              0.0
I_gamma                      100.0
I_gamma Unc                    0.0
Mult                            M1
B_exp (W.u.)                  10.9
B_exp Unc 

C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

-------------------------------------------
Excluding transition i = 87
Matched # initial states: 1
Matched # final   states: 0
Matched # Ji             : 11
Matched # Jf             : 27
Matched # Ei             : 1
Matched # Ef             : 0
    Nucleus Elevel(keV)    J   E(keV) dE(keV)  Include  twoJ Element   A  n
678    21Ne  4684.53 15  3/2  4684.53    15.0     True     3      Ne  21  2
Empty DataFrame
Columns: [Nucleus, Elevel(keV), J, E(keV), dE(keV), Include, twoJ, Element, A, n]
Index: []
A                           21
Element                     Ne
Z                           10
N                           11
Energy (keV)           4684.53
Energy Unc (keV)          0.15
Ji                         3/2
Thalf                  11 FS 3
E_gamma (keV)          4683.94
E_gamma Unc (keV)         0.25
I_gamma                   59.0
I_gamma Unc                2.1
Mult                        M1
B_exp (W.u.)            0.0071
B_exp Unc (W.u.)         0.002
twoJi                        

C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

-------------------------------------------
Excluding transition i = 241
Matched # initial states: 1
Matched # final   states: 0
Matched # Ji             : 7
Matched # Jf             : 20
Matched # Ei             : 1
Matched # Ef             : 0
     Nucleus Elevel(keV)  J   E(keV) dE(keV)  Include  twoJ Element   A  n
2227    28Al   2201.43 3  1  2201.43     3.0     True     2      Al  28  3
Empty DataFrame
Columns: [Nucleus, Elevel(keV), J, E(keV), dE(keV), Include, twoJ, Element, A, n]
Index: []
A                           28
Element                     Al
Z                           13
N                           15
Energy (keV)           2201.43
Energy Unc (keV)          0.03
Ji                           1
Thalf                  32 FS 2
E_gamma (keV)           1229.8
E_gamma Unc (keV)          0.4
I_gamma                   20.0
I_gamma Unc                3.0
Mult                      [M1]
B_exp (W.u.)             0.054
B_exp Unc (W.u.)          0.01
twoJi                        2


C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\MainUser\anaconda3\envs\uq\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

In [28]:
df_levels['Nucleus']==df['Nucleus'].loc[i]

0       False
1       False
2       False
3       False
4       False
        ...  
7876    False
7877    False
7878    False
7879    False
7880    False
Name: Nucleus, Length: 4039, dtype: bool

In [29]:
df[df['Include']==False]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),...,Thalf (sec),Thalf Unc (sec),B_exp (mu_N^2),B_exp Unc (mu_N^2),Nucleus,Jf,twoJf,ni,nf,Include
7,18,F,9,9,1041.55,0.08,0,1.77 FS 0.31,1041.00,0.00,...,1.770000e-15,3.100000e-16,19.516375,3.760036,18F,None,None,None,None,False
20,18,F,9,9,5603.38,0.27,1,43.3 EV 1.6,4561.00,0.00,...,1.053648e-20,2.851434e-19,0.016114,0.005371,18F,None,None,None,None,False
21,18,F,9,9,5603.38,0.27,1,43.3 EV 1.6,2541.00,0.00,...,1.053648e-20,2.851434e-19,1.969542,0.358099,18F,None,None,None,None,False
33,18,F,9,9,6310.50,0.80,3,0.95 KEV 0.14,2472.00,0.00,...,4.802416e-19,3.258782e-18,0.044762,0.016114,18F,None,None,None,None,False
36,18,F,9,9,6310.50,0.80,3,0.95 KEV 0.14,3788.00,0.00,...,4.802416e-19,3.258782e-18,0.010743,0.003223,18F,None,None,None,None,False
39,18,F,9,9,6385.50,1.70,2,0.49 KEV 0.09,2547.00,0.00,...,9.310806e-19,5.069216e-18,0.322289,0.161144,18F,None,None,None,None,False
87,21,Ne,10,11,4684.53,0.15,3/2,11 FS 3,4683.94,0.25,...,1.100000e-14,3.000000e-15,0.012713,0.003581,21Ne,None,None,None,None,False
90,21,Na,11,10,2423.80,0.40,1/2,1.7 FS 0.4,2423.30,0.50,...,1.700000e-15,4.000000e-16,1.629349,0.393908,21Na,None,None,None,None,False
92,22,Na,11,11,1936.90,0.20,1,13 FS 4,1280.50,1.00,...,1.300000e-14,4.000000e-15,1.432394,0.447623,22Na,None,None,None,None,False
104,23,Na,11,12,3914.60,0.40,5/2,6.9 FS 1.4,932.00,1.00,...,6.900000e-15,1.400000e-15,0.162935,0.037600,23Na,None,None,None,None,False


In [32]:
df[df['Include']==True]

,A,Element,Z,N,Energy (keV),Energy Unc (keV),Ji,Thalf,E_gamma (keV),E_gamma Unc (keV),...,Thalf (sec),Thalf Unc (sec),B_exp (mu_N^2),B_exp Unc (mu_N^2),Nucleus,Jf,twoJf,ni,nf,Include
0,16,O,8,8,9844.50,0.50,2,0.62 KEV 0.1,2927.1,0.8,...,7.358540e-19,4.562295e-18,0.007520,0.001432,16O,2,4,2,1,True
1,18,O,8,10,3920.44,0.14,2,18.4 FS 2,1938.0,0.0,...,1.840000e-14,2.000000e-15,0.250669,0.035810,18O,2,4,2,1,True
2,18,O,8,10,5254.80,0.90,2,7.0 FS 0.3,1334.0,0.0,...,7.000000e-15,3.000000e-16,0.198745,0.014324,18O,2,4,3,2,True
4,18,O,8,10,8213.00,4.00,2,1.0 KEV 0.8,4293.0,0.0,...,4.562295e-19,5.702868e-19,0.012892,0.005371,18O,2,4,4,2,True
5,19,O,8,11,96.00,0.50,3/2,1.39 NS 0.05,96.0,0.0,...,1.390000e-09,5.000000e-11,0.157563,0.068039,19O,5/2,5,1,1,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,35,Cl,17,18,3967.50,0.23,1/2,10 FS 3,2748.5,0.6,...,1.000000e-14,3.000000e-15,0.141449,0.044762,35Cl,1/2,1,2,1,True
287,38,K,19,19,458.53,0.16,1,7.0 PS 0.6,328.1,0.2,...,7.000000e-12,6.000000e-13,0.157563,0.016114,38K,0,0,1,1,True
288,38,K,19,19,1697.65,0.25,1,48 FS 10,1567.4,0.2,...,4.800000e-14,1.000000e-14,0.213069,0.057296,38K,0,0,2,1,True
293,39,K,19,20,4095.60,0.40,1/2,60 FS 10,1572.5,0.3,...,6.000000e-14,1.000000e-14,0.143239,0.032229,39K,1/2,1,2,1,True


In [30]:
df_output = df[df['Include']==True]

In [31]:
df_output.to_csv('cleaned_sd_M1_data_v3.csv',index=False)